In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import re
import xgboost

from pandas.api.types import CategoricalDtype

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from statsmodels.graphics.tsaplots import plot_pacf
from xgboost import XGBRegressor
#import lightgbm

In [4]:
all_csv = pd.read_csv('cleaned.csv.gz',
                 dtype = {
                     'store_nbr' : 'category',
                     'family' : 'category',
                     'sales': 'float',
                     'city': 'category',
                     'state': 'category',
                     'type': 'category',
                     'holiday_type': 'category',
                     'holiday_transferred': 'category'
                 },
                  parse_dates=['date'])
all_csv['date'] = pd.to_datetime(all_csv['date']).dt.to_period('D')

In [5]:
all = all_csv.copy()  # we can start experimenting from here without reloading the csv file

In [6]:
# this is for experimentation

filter_by_stores = None  # note: please use string here (unlike Mine.ipynb)
filter_by_family = None
filter_by_dates = None

#filter_by_stores = ['1', '2']  # note: please use string here (unlike Mine.ipynb)
#filter_by_family = ['DAIRY', 'PRODUCE']
#filter_by_family = ['']
#filter_by_dates = '2014-06-05'

In [7]:
if filter_by_dates == None:
    train_start_date = '2013-01-01'
else:
    train_start_date = filter_by_dates
train_end_date = '2017-08-15'
test_start_date = '2017-08-16'
test_end_date = '2017-08-31'

In [8]:
if filter_by_family != None:
    all = all[all['family'].isin(filter_by_family)]
if filter_by_stores != None:
    all = all[all['store_nbr'].isin(filter_by_stores)]
if filter_by_dates != None:
    all = all[all['date'] >= filter_by_dates]

In [9]:
all.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3036528 entries, 0 to 3036527
Data columns (total 38 columns):
 #   Column               Dtype    
---  ------               -----    
 0   date                 period[D]
 1   store_nbr            category 
 2   family               category 
 3   sales                float64  
 4   onpromotion          int64    
 5   sales_lag_01         float64  
 6   sales_lag_02         float64  
 7   sales_lag_03         float64  
 8   sales_lag_04         float64  
 9   sales_lag_05         float64  
 10  sales_lag_06         float64  
 11  sales_lag_07         float64  
 12  sales_lag_08         float64  
 13  sales_lag_09         float64  
 14  sales_lag_10         float64  
 15  sales_lag_11         float64  
 16  sales_lag_12         float64  
 17  sales_lag_13         float64  
 18  sales_lag_14         float64  
 19  sales_lag_15         float64  
 20  sales_lag_16         float64  
 21  sales_lag_17         float64  
 22  sales_lag_18      

## One Hot Encoding

In [10]:
def one_hot_encode(df):
    return pd.get_dummies(data=df, columns=['store_nbr', 'family', 'city', 'state', 'type',
                                     'cluster', 'holiday_type', 'holiday_transferred', 'weekday'])    

In [11]:
all_ohe = one_hot_encode(all)
all_ohe = all_ohe.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))  # remove bad char in column names

X = all_ohe[all_ohe['date'] <= train_end_date]
X = X.drop(['sales'], axis=1)
y = all_ohe[['date', 'sales']][all_ohe['date'] <= train_end_date]
y.set_index('date', inplace=True)

X_test = all_ohe[all_ohe['date'] >= test_start_date]
X_test = X_test.drop(['sales'], axis=1)

X.drop('date', axis=1, inplace=True)
X_test.drop('date', axis=1, inplace=True)
y.set_index(X.index, inplace=True)
X.columns.size
# X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=1)

In [ ]:
y.columns

## PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=180)
principalComponents = pca.fit_transform(X)
principalDf = pd.DataFrame(data = principalComponents)

X_train, X_val, y_train, y_val = train_test_split(principalDf, y, random_state=1,stratify='sales')

X_train.columns.size

## Experiment I -- Linear Regression

In [10]:
run_experiment_I = True
if run_experiment_I:
    lr = LinearRegression()
    lr.fit(X_train, y_train)

In [11]:
if run_experiment_I:
    y_pred_train = lr.predict(X_train)
    y_pred_train[y_pred_train < 0] = 0
    y_pred_val = lr.predict(X_val)
    y_pred_val[y_pred_val < 0] = 0

    print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
    print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))
    print("RMS log-error train (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_train), np.expm1(y_pred_train))))
    print("RMS log-error val (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))))

RMS log-error train:  0.20928922205788442
RMS log-error val:  0.231758418939929
RMS log-error train (actual):  0.9087185771821745
RMS log-error val (actual):  0.9985451307445373


## Experiment II -- Lightgbm

In [12]:
lgb_params = {
    'boosting_type' : 'gbdt',  # gradient boosting decision tree
    'early_stopping_rounds': 200,
    'force_col_wise': True,
    'learning_rate': 0.1,
    'max_depth': 10,
    'metric': 'mse',  # mean square error
    'num_iterations': 5000,
    'num_leaves': 10,
    'random_state': 1,
    'verbose': 0
}

run_experiment_II = True  # should be true
if run_experiment_II:
    X_train_lgb = lightgbm.Dataset(data=X_train, label=y_train, feature_name='auto')
    X_val_lgb = lightgbm.Dataset(data=X_val, label=y_val, reference=X_train_lgb, feature_name='auto')

In [13]:
if run_experiment_II:
    lgb = lightgbm.train(
        params=lgb_params, 
        train_set=X_train_lgb,
        valid_sets=[X_train_lgb, X_val_lgb],
    )

[1]	training's l2: 3.08025	valid_1's l2: 3.32536
Training until validation scores don't improve for 200 rounds
[2]	training's l2: 2.64088	valid_1's l2: 2.8625
[3]	training's l2: 2.28494	valid_1's l2: 2.49129
[4]	training's l2: 1.99155	valid_1's l2: 2.18464
[5]	training's l2: 1.75269	valid_1's l2: 1.93936
[6]	training's l2: 1.55293	valid_1's l2: 1.73581
[7]	training's l2: 1.38871	valid_1's l2: 1.56638
[8]	training's l2: 1.2528	valid_1's l2: 1.43213
[9]	training's l2: 1.14397	valid_1's l2: 1.32266
[10]	training's l2: 1.05567	valid_1's l2: 1.23178
[11]	training's l2: 0.982369	valid_1's l2: 1.16004
[12]	training's l2: 0.919926	valid_1's l2: 1.10163
[13]	training's l2: 0.869092	valid_1's l2: 1.05099
[14]	training's l2: 0.826757	valid_1's l2: 1.01009
[15]	training's l2: 0.788786	valid_1's l2: 0.980336
[16]	training's l2: 0.753583	valid_1's l2: 0.945958
[17]	training's l2: 0.724031	valid_1's l2: 0.917791
[18]	training's l2: 0.69783	valid_1's l2: 0.900712
[19]	training's l2: 0.674961	valid_1's

/usr/local/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[171]	training's l2: 0.199387	valid_1's l2: 0.764011
[172]	training's l2: 0.197458	valid_1's l2: 0.763854
[173]	training's l2: 0.1966	valid_1's l2: 0.761488
[174]	training's l2: 0.195455	valid_1's l2: 0.761336
[175]	training's l2: 0.194324	valid_1's l2: 0.763025
[176]	training's l2: 0.193571	valid_1's l2: 0.763779
[177]	training's l2: 0.192727	valid_1's l2: 0.763965
[178]	training's l2: 0.191959	valid_1's l2: 0.764188
[179]	training's l2: 0.191085	valid_1's l2: 0.76299
[180]	training's l2: 0.190247	valid_1's l2: 0.761166
[181]	training's l2: 0.189809	valid_1's l2: 0.761296
[182]	training's l2: 0.188997	valid_1's l2: 0.761071
[183]	training's l2: 0.18798	valid_1's l2: 0.760581
[184]	training's l2: 0.186727	valid_1's l2: 0.760571
[185]	training's l2: 0.185686	valid_1's l2: 0.760414
[186]	training's l2: 0.185	valid_1's l2: 0.760588
[187]	training's l2: 0.183842	valid_1's l2: 0.76059
[188]	training's l2: 0.182729	valid_1's l2: 0.760463
[189]	training's l2: 0.181973	valid_1's l2: 0.761721
[

[374]	training's l2: 0.0847432	valid_1's l2: 0.771207
[375]	training's l2: 0.0843536	valid_1's l2: 0.77109
[376]	training's l2: 0.0840959	valid_1's l2: 0.770744
[377]	training's l2: 0.0837883	valid_1's l2: 0.771273
[378]	training's l2: 0.0834566	valid_1's l2: 0.771106
[379]	training's l2: 0.0833779	valid_1's l2: 0.771085
[380]	training's l2: 0.0832138	valid_1's l2: 0.770778
[381]	training's l2: 0.082857	valid_1's l2: 0.770971
[382]	training's l2: 0.0827012	valid_1's l2: 0.771202
[383]	training's l2: 0.0824927	valid_1's l2: 0.771702
[384]	training's l2: 0.0821722	valid_1's l2: 0.771245
[385]	training's l2: 0.0817519	valid_1's l2: 0.770176
[386]	training's l2: 0.0814709	valid_1's l2: 0.770151
[387]	training's l2: 0.0810742	valid_1's l2: 0.769944
[388]	training's l2: 0.0809308	valid_1's l2: 0.769869
[389]	training's l2: 0.0807303	valid_1's l2: 0.769868
[390]	training's l2: 0.0804766	valid_1's l2: 0.769616
[391]	training's l2: 0.0801384	valid_1's l2: 0.768602
[392]	training's l2: 0.0797511

In [14]:
if run_experiment_II:
    y_pred_train = lgb.predict(X_train, num_iteration=lgb.best_iteration)
    y_pred_train[y_pred_train < 0] = 0
    y_pred_val = lgb.predict(X_val, num_iteration=lgb.best_iteration)
    y_pred_val[y_pred_val < 0] = 0

    print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
    print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))
    print("RMS log-error train (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_train), np.expm1(y_pred_train))))
    print("RMS log-error val (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))))

RMS log-error train:  0.09706488341239258
RMS log-error val:  0.2018445806471314
RMS log-error train (actual):  0.3953365923801808
RMS log-error val (actual):  0.8708139540731853


## Experiment III -- Random Forest (Too Slow)

In [11]:
run_experiment_III = True
if run_experiment_III:
    random_forest = RandomForestRegressor(random_state=1)
    random_forest.fit(X_train, y_train.values.ravel())

KeyboardInterrupt: 

In [16]:
if run_experiment_III:
    y_pred_train = random_forest.predict(X_train)
    y_pred_train[y_pred_train < 0] = 0
    y_pred_val = random_forest.predict(X_val)
    y_pred_val[y_pred_val < 0] = 0

    print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
    print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))
    print("RMS log-error train (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_train), np.expm1(y_pred_train))))
    print("RMS log-error val (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))))

RMS log-error train:  0.07991230493098243
RMS log-error val:  0.198402429075905
RMS log-error train (actual):  0.29627903576647857
RMS log-error val (actual):  0.8535536261784764


## Experiment IV -- XGBoost

In [12]:
param = {
      "max_depth": [6,10],
      "n_estimators": [80,100],
#        "early_stopping_rounds"=[50,55,60],
#       "learning_rate": [0.01,0.1],
      #'colsample_bylevel': [0.1,0.5,1]
      
  }

In [11]:
run_experiment_IV = True
if run_experiment_IV:
    xgb = xgboost.XGBRegressor()
# xgb.fit(X_train, y_train,
#         eval_set=[(X_train, y_train), (X_val, y_val)],
#         verbose=True)

In [12]:
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param,
    scoring = 'neg_mean_squared_error',
    n_jobs = -1,
    cv = 2,
    verbose=True
)

In [ ]:
grid_search.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        verbose=True)

Fitting 2 folds for each of 4 candidates, totalling 8 fits


In [18]:
if run_experiment_IV:
    y_pred_train = xgb.predict(X_train)
    y_pred_train[y_pred_train < 0] = 0
    y_pred_val = xgb.predict(X_val)
    y_pred_val[y_pred_val < 0] = 0

    print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
    print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))
    print("RMS log-error train (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_train), np.expm1(y_pred_train))))
    print("RMS log-error val (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))))

RMS log-error train:  0.09657077521238491
RMS log-error val:  0.19491349654872678
RMS log-error train (actual):  0.3971298372479599
RMS log-error val (actual):  0.8530671459927017


In [13]:
!pip install -q catboost

In [14]:
from catboost import CatBoostRegressor

In [31]:
cat = CatBoostRegressor()

In [33]:
param = {
      "max_depth": [6,8],
      "n_estimators": [1000,1400,1800],
#       "learning_rate": [0.01,0.1],
      #'colsample_bylevel': [0.1,0.5,1]
      
  }

grid_search = GridSearchCV(
    estimator=cat,
    param_grid=param,
    scoring = 'neg_mean_squared_error',
    n_jobs = -1,
    cv = 3,
    verbose=True
)

grid_search.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        verbose=True,early_stopping_rounds=80)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Learning rate set to 0.112204
0:	learn: 2.1915463	test: 2.1906993	test1: 2.1930827	best: 2.1930827 (0)	total: 1.05s	remaining: 31m 22s
1:	learn: 1.9675788	test: 1.9667727	test1: 1.9690645	best: 1.9690645 (1)	total: 1.53s	remaining: 22m 53s
2:	learn: 1.7675389	test: 1.7667902	test1: 1.7690854	best: 1.7690854 (2)	total: 1.9s	remaining: 18m 58s
3:	learn: 1.5908671	test: 1.5901378	test1: 1.5923298	best: 1.5923298 (3)	total: 2.46s	remaining: 18m 23s
4:	learn: 1.4344965	test: 1.4338167	test1: 1.4360755	best: 1.4360755 (4)	total: 2.98s	remaining: 17m 48s
5:	learn: 1.2963956	test: 1.2957473	test1: 1.2979073	best: 1.2979073 (5)	total: 3.43s	remaining: 17m 5s
6:	learn: 1.1749485	test: 1.1743265	test1: 1.1765135	best: 1.1765135 (6)	total: 3.95s	remaining: 16m 50s
7:	learn: 1.0680517	test: 1.0674611	test1: 1.0696519	best: 1.0696519 (7)	total: 4.49s	remaining: 16m 46s
8:	learn: 0.9743743	test: 0.9737934	test1: 0.9759642	best: 0.9759642 (8)

Learning rate set to 0.112204
0:	learn: 2.1910289	test: 2.1913278	test1: 2.1936813	best: 2.1936813 (0)	total: 724ms	remaining: 21m 42s
1:	learn: 1.9662299	test: 1.9665386	test1: 1.9688440	best: 1.9688440 (1)	total: 1.23s	remaining: 18m 24s
2:	learn: 1.7663688	test: 1.7666798	test1: 1.7690124	best: 1.7690124 (2)	total: 1.74s	remaining: 17m 21s
3:	learn: 1.5889790	test: 1.5892966	test1: 1.5916083	best: 1.5916083 (3)	total: 2.16s	remaining: 16m 9s
4:	learn: 1.4325114	test: 1.4328432	test1: 1.4351338	best: 1.4351338 (4)	total: 2.66s	remaining: 15m 55s
5:	learn: 1.2947897	test: 1.2951177	test1: 1.2973213	best: 1.2973213 (5)	total: 3.22s	remaining: 16m 2s
6:	learn: 1.1727692	test: 1.1731046	test1: 1.1752720	best: 1.1752720 (6)	total: 3.71s	remaining: 15m 50s
7:	learn: 1.0655296	test: 1.0658748	test1: 1.0679359	best: 1.0679359 (7)	total: 4.23s	remaining: 15m 47s
8:	learn: 0.9715822	test: 0.9719314	test1: 0.9739451	best: 0.9739451 (8)	total: 4.81s	remaining: 15m 56s
9:	learn: 0.8895623	test: 0

Learning rate set to 0.130793
0:	learn: 2.1499361	test: 2.1490972	test1: 2.1514683	best: 2.1514683 (0)	total: 763ms	remaining: 17m 47s
1:	learn: 1.8947145	test: 1.8939226	test1: 1.8961883	best: 1.8961883 (1)	total: 1.34s	remaining: 15m 38s
2:	learn: 1.6720267	test: 1.6712836	test1: 1.6734705	best: 1.6734705 (2)	total: 1.93s	remaining: 14m 57s
3:	learn: 1.4789662	test: 1.4782765	test1: 1.4803941	best: 1.4803941 (3)	total: 2.36s	remaining: 13m 43s
4:	learn: 1.3116991	test: 1.3110833	test1: 1.3131953	best: 1.3131953 (4)	total: 2.88s	remaining: 13m 24s
5:	learn: 1.1680265	test: 1.1674545	test1: 1.1694188	best: 1.1694188 (5)	total: 3.45s	remaining: 13m 20s
6:	learn: 1.0450964	test: 1.0445375	test1: 1.0464578	best: 1.0464578 (6)	total: 3.86s	remaining: 12m 47s
7:	learn: 0.9390917	test: 0.9385483	test1: 0.9403505	best: 0.9403505 (7)	total: 4.3s	remaining: 12m 29s
8:	learn: 0.8488558	test: 0.8483595	test1: 0.8500663	best: 0.8500663 (8)	total: 4.8s	remaining: 12m 21s
9:	learn: 0.7725933	test: 0

Learning rate set to 0.130793
0:	learn: 2.1495335	test: 2.1498361	test1: 2.1521722	best: 2.1521722 (0)	total: 653ms	remaining: 15m 12s
1:	learn: 1.8932755	test: 1.8935900	test1: 1.8958734	best: 1.8958734 (1)	total: 1.16s	remaining: 13m 32s
2:	learn: 1.6700754	test: 1.6703618	test1: 1.6726646	best: 1.6726646 (2)	total: 1.67s	remaining: 12m 58s
3:	learn: 1.4772100	test: 1.4775300	test1: 1.4797573	best: 1.4797573 (3)	total: 2.19s	remaining: 12m 43s
4:	learn: 1.3104339	test: 1.3107751	test1: 1.3130184	best: 1.3130184 (4)	total: 2.68s	remaining: 12m 28s
5:	learn: 1.1668938	test: 1.1672649	test1: 1.1694653	best: 1.1694653 (5)	total: 3.14s	remaining: 12m 9s
6:	learn: 1.0439800	test: 1.0443607	test1: 1.0465014	best: 1.0465014 (6)	total: 3.61s	remaining: 11m 58s
7:	learn: 0.9384678	test: 0.9388595	test1: 0.9409543	best: 0.9409543 (7)	total: 4.08s	remaining: 11m 49s
8:	learn: 0.8485246	test: 0.8489217	test1: 0.8509456	best: 0.8509456 (8)	total: 4.49s	remaining: 11m 33s
9:	learn: 0.7721177	test: 

Learning rate set to 0.160592
0:	learn: 2.0834967	test: 2.0840074	test1: 2.0863740	best: 2.0863740 (0)	total: 634ms	remaining: 10m 33s
1:	learn: 1.7792270	test: 1.7796083	test1: 1.7818620	best: 1.7818620 (1)	total: 1.27s	remaining: 10m 33s
2:	learn: 1.5238948	test: 1.5242220	test1: 1.5264405	best: 1.5264405 (2)	total: 1.75s	remaining: 9m 41s
3:	learn: 1.3122231	test: 1.3125149	test1: 1.3146053	best: 1.3146053 (3)	total: 2.14s	remaining: 8m 53s
4:	learn: 1.1357271	test: 1.1359873	test1: 1.1380379	best: 1.1380379 (4)	total: 2.56s	remaining: 8m 29s
5:	learn: 0.9903516	test: 0.9905898	test1: 0.9925707	best: 0.9925707 (5)	total: 3.12s	remaining: 8m 37s
6:	learn: 0.8716704	test: 0.8718611	test1: 0.8738265	best: 0.8738265 (6)	total: 3.58s	remaining: 8m 28s
7:	learn: 0.7741515	test: 0.7743144	test1: 0.7761396	best: 0.7761396 (7)	total: 4.08s	remaining: 8m 26s
8:	learn: 0.6960642	test: 0.6962076	test1: 0.6979738	best: 0.6979738 (8)	total: 4.6s	remaining: 8m 26s
9:	learn: 0.6339449	test: 0.63407

...

1683:	learn: 0.3397965	test: 0.3397965	test1: 0.3575828	best: 0.3575800 (1682)	total: 3m 33s	remaining: 14.7s
1684:	learn: 0.3397800	test: 0.3397800	test1: 0.3575723	best: 0.3575723 (1684)	total: 3m 33s	remaining: 14.6s
1685:	learn: 0.3397619	test: 0.3397619	test1: 0.3575668	best: 0.3575668 (1685)	total: 3m 33s	remaining: 14.4s
1686:	learn: 0.3397464	test: 0.3397464	test1: 0.3575564	best: 0.3575564 (1686)	total: 3m 33s	remaining: 14.3s
1687:	learn: 0.3397317	test: 0.3397317	test1: 0.3575464	best: 0.3575464 (1687)	total: 3m 33s	remaining: 14.2s
1688:	learn: 0.3397219	test: 0.3397219	test1: 0.3575433	best: 0.3575433 (1688)	total: 3m 33s	remaining: 14.1s
1689:	learn: 0.3397124	test: 0.3397124	test1: 0.3575415	best: 0.3575415 (1689)	total: 3m 33s	remaining: 13.9s
1690:	learn: 0.3396984	test: 0.3396984	test1: 0.3575390	best: 0.3575390 (1690)	total: 3m 34s	remaining: 13.8s
1691:	learn: 0.3396810	test: 0.3396810	test1: 0.3575279	best: 0.3575279 (1691)	total: 3m 34s	remaining: 13.7s
1692:	lear

1759:	learn: 0.3387564	test: 0.3387564	test1: 0.3570652	best: 0.3570652 (1759)	total: 3m 42s	remaining: 5.06s
1760:	learn: 0.3387475	test: 0.3387475	test1: 0.3570588	best: 0.3570588 (1760)	total: 3m 42s	remaining: 4.94s
1761:	learn: 0.3387422	test: 0.3387422	test1: 0.3570550	best: 0.3570550 (1761)	total: 3m 43s	remaining: 4.81s
1762:	learn: 0.3387333	test: 0.3387333	test1: 0.3570453	best: 0.3570453 (1762)	total: 3m 43s	remaining: 4.68s
1763:	learn: 0.3387197	test: 0.3387197	test1: 0.3570368	best: 0.3570368 (1763)	total: 3m 43s	remaining: 4.56s
1764:	learn: 0.3387062	test: 0.3387062	test1: 0.3570283	best: 0.3570283 (1764)	total: 3m 43s	remaining: 4.43s
1765:	learn: 0.3386898	test: 0.3386898	test1: 0.3570220	best: 0.3570220 (1765)	total: 3m 43s	remaining: 4.3s
1766:	learn: 0.3386756	test: 0.3386756	test1: 0.3570157	best: 0.3570157 (1766)	total: 3m 43s	remaining: 4.18s
1767:	learn: 0.3386605	test: 0.3386605	test1: 0.3570073	best: 0.3570073 (1767)	total: 3m 43s	remaining: 4.05s
1768:	learn

GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostRegressor object at 0x168e9a710>,
             n_jobs=-1,
             param_grid={'max_depth': [6, 8],
                         'n_estimators': [1000, 1400, 1800]},
             scoring='neg_mean_squared_error', verbose=True)

1582:	learn: 0.3396141	test: 0.3467823	test1: 0.3611809	best: 0.3611809 (1582)	total: 11m 16s	remaining: 1m 32s
1583:	learn: 0.3395936	test: 0.3467658	test1: 0.3611741	best: 0.3611741 (1583)	total: 11m 16s	remaining: 1m 32s
1584:	learn: 0.3395800	test: 0.3467552	test1: 0.3611712	best: 0.3611712 (1584)	total: 11m 17s	remaining: 1m 31s
1585:	learn: 0.3395565	test: 0.3467351	test1: 0.3611556	best: 0.3611556 (1585)	total: 11m 17s	remaining: 1m 31s
1586:	learn: 0.3395385	test: 0.3467205	test1: 0.3611463	best: 0.3611463 (1586)	total: 11m 17s	remaining: 1m 30s
1587:	learn: 0.3395163	test: 0.3467016	test1: 0.3611334	best: 0.3611334 (1587)	total: 11m 17s	remaining: 1m 30s
1588:	learn: 0.3395022	test: 0.3466886	test1: 0.3611232	best: 0.3611232 (1588)	total: 11m 17s	remaining: 1m 29s
1589:	learn: 0.3394877	test: 0.3466768	test1: 0.3611115	best: 0.3611115 (1589)	total: 11m 17s	remaining: 1m 29s
1590:	learn: 0.3394717	test: 0.3466644	test1: 0.3611071	best: 0.3611071 (1590)	total: 11m 17s	remaining:

1581:	learn: 0.3395895	test: 0.3471047	test1: 0.3608209	best: 0.3608209 (1581)	total: 11m 13s	remaining: 1m 32s
1582:	learn: 0.3395826	test: 0.3470998	test1: 0.3608195	best: 0.3608195 (1582)	total: 11m 13s	remaining: 1m 32s
1583:	learn: 0.3395753	test: 0.3470932	test1: 0.3608174	best: 0.3608174 (1583)	total: 11m 13s	remaining: 1m 31s
1584:	learn: 0.3395652	test: 0.3470862	test1: 0.3608156	best: 0.3608156 (1584)	total: 11m 13s	remaining: 1m 31s
1585:	learn: 0.3395540	test: 0.3470779	test1: 0.3608165	best: 0.3608156 (1584)	total: 11m 13s	remaining: 1m 30s
1586:	learn: 0.3395409	test: 0.3470669	test1: 0.3608113	best: 0.3608113 (1586)	total: 11m 13s	remaining: 1m 30s
1587:	learn: 0.3395157	test: 0.3470477	test1: 0.3608005	best: 0.3608005 (1587)	total: 11m 14s	remaining: 1m 29s
1588:	learn: 0.3394958	test: 0.3470323	test1: 0.3607870	best: 0.3607870 (1588)	total: 11m 14s	remaining: 1m 29s
1589:	learn: 0.3394875	test: 0.3470271	test1: 0.3607843	best: 0.3607843 (1589)	total: 11m 14s	remaining:

In [23]:
# cat.fit(X_train,y_train,plot=True,eval_set=[(X_train, y_train), (X_val, y_val)], verbose=True,early_stopping_rounds=50,)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.171147
0:	learn: 2.0585786	test: 2.0585786	test1: 2.0608291	best: 2.0608291 (0)	total: 115ms	remaining: 1m 54s
1:	learn: 1.7397478	test: 1.7397478	test1: 1.7418322	best: 1.7418322 (1)	total: 220ms	remaining: 1m 49s
2:	learn: 1.4770137	test: 1.4770137	test1: 1.4789618	best: 1.4789618 (2)	total: 315ms	remaining: 1m 44s
3:	learn: 1.2601855	test: 1.2601855	test1: 1.2621024	best: 1.2621024 (3)	total: 414ms	remaining: 1m 43s
4:	learn: 1.0813202	test: 1.0813202	test1: 1.0832465	best: 1.0832465 (4)	total: 510ms	remaining: 1m 41s
5:	learn: 0.9361416	test: 0.9361416	test1: 0.9380245	best: 0.9380245 (5)	total: 600ms	remaining: 1m 39s
6:	learn: 0.8197486	test: 0.8197486	test1: 0.8215290	best: 0.8215290 (6)	total: 696ms	remaining: 1m 38s
7:	learn: 0.7263216	test: 0.7263216	test1: 0.7280233	best: 0.7280233 (7)	total: 793ms	remaining: 1m 38s
8:	learn: 0.6533236	test: 0.6533236	test1: 0.6549661	best: 0.6549661 (8)	total: 891ms	remaining: 1m 38s
9:	learn: 0.5967435	test: 0.596743

78:	learn: 0.4017672	test: 0.4017672	test1: 0.4034496	best: 0.4034496 (78)	total: 6.81s	remaining: 1m 19s
79:	learn: 0.4014503	test: 0.4014503	test1: 0.4031269	best: 0.4031269 (79)	total: 6.91s	remaining: 1m 19s
80:	learn: 0.4012310	test: 0.4012310	test1: 0.4029307	best: 0.4029307 (80)	total: 7.01s	remaining: 1m 19s
81:	learn: 0.4010170	test: 0.4010170	test1: 0.4027060	best: 0.4027060 (81)	total: 7.09s	remaining: 1m 19s
82:	learn: 0.4008356	test: 0.4008356	test1: 0.4025315	best: 0.4025315 (82)	total: 7.17s	remaining: 1m 19s
83:	learn: 0.4006785	test: 0.4006785	test1: 0.4023798	best: 0.4023798 (83)	total: 7.24s	remaining: 1m 18s
84:	learn: 0.4005317	test: 0.4005317	test1: 0.4022426	best: 0.4022426 (84)	total: 7.32s	remaining: 1m 18s
85:	learn: 0.4003412	test: 0.4003412	test1: 0.4020408	best: 0.4020408 (85)	total: 7.42s	remaining: 1m 18s
86:	learn: 0.4001776	test: 0.4001776	test1: 0.4018774	best: 0.4018774 (86)	total: 7.5s	remaining: 1m 18s
87:	learn: 0.3999426	test: 0.3999426	test1: 0.4

156:	learn: 0.3901546	test: 0.3901546	test1: 0.3925746	best: 0.3925746 (156)	total: 13.5s	remaining: 1m 12s
157:	learn: 0.3900911	test: 0.3900911	test1: 0.3925250	best: 0.3925250 (157)	total: 13.6s	remaining: 1m 12s
158:	learn: 0.3899790	test: 0.3899790	test1: 0.3924262	best: 0.3924262 (158)	total: 13.7s	remaining: 1m 12s
159:	learn: 0.3898708	test: 0.3898708	test1: 0.3923508	best: 0.3923508 (159)	total: 13.8s	remaining: 1m 12s
160:	learn: 0.3896633	test: 0.3896633	test1: 0.3921401	best: 0.3921401 (160)	total: 13.9s	remaining: 1m 12s
161:	learn: 0.3895731	test: 0.3895731	test1: 0.3920535	best: 0.3920535 (161)	total: 13.9s	remaining: 1m 12s
162:	learn: 0.3895117	test: 0.3895117	test1: 0.3919982	best: 0.3919982 (162)	total: 14s	remaining: 1m 12s
163:	learn: 0.3894266	test: 0.3894266	test1: 0.3919148	best: 0.3919148 (163)	total: 14.1s	remaining: 1m 11s
164:	learn: 0.3893103	test: 0.3893103	test1: 0.3918096	best: 0.3918096 (164)	total: 14.2s	remaining: 1m 11s
165:	learn: 0.3892322	test: 0.

234:	learn: 0.3834828	test: 0.3834828	test1: 0.3870266	best: 0.3870266 (234)	total: 20.2s	remaining: 1m 5s
235:	learn: 0.3834223	test: 0.3834223	test1: 0.3869704	best: 0.3869704 (235)	total: 20.3s	remaining: 1m 5s
236:	learn: 0.3833651	test: 0.3833651	test1: 0.3869134	best: 0.3869134 (236)	total: 20.3s	remaining: 1m 5s
237:	learn: 0.3833003	test: 0.3833003	test1: 0.3868971	best: 0.3868971 (237)	total: 20.5s	remaining: 1m 5s
238:	learn: 0.3832657	test: 0.3832657	test1: 0.3868873	best: 0.3868873 (238)	total: 20.5s	remaining: 1m 5s
239:	learn: 0.3831956	test: 0.3831956	test1: 0.3868302	best: 0.3868302 (239)	total: 20.6s	remaining: 1m 5s
240:	learn: 0.3831378	test: 0.3831378	test1: 0.3867768	best: 0.3867768 (240)	total: 20.7s	remaining: 1m 5s
241:	learn: 0.3830526	test: 0.3830526	test1: 0.3866929	best: 0.3866929 (241)	total: 20.8s	remaining: 1m 5s
242:	learn: 0.3830041	test: 0.3830041	test1: 0.3866433	best: 0.3866433 (242)	total: 20.9s	remaining: 1m 5s
243:	learn: 0.3829510	test: 0.3829510

...

623:	learn: 0.3681140	test: 0.3681140	test1: 0.3752296	best: 0.3752296 (623)	total: 53.2s	remaining: 32.1s
624:	learn: 0.3680918	test: 0.3680918	test1: 0.3752109	best: 0.3752109 (624)	total: 53.3s	remaining: 32s
625:	learn: 0.3680566	test: 0.3680566	test1: 0.3751985	best: 0.3751985 (625)	total: 53.4s	remaining: 31.9s
626:	learn: 0.3680357	test: 0.3680357	test1: 0.3751982	best: 0.3751982 (626)	total: 53.5s	remaining: 31.8s
627:	learn: 0.3680143	test: 0.3680143	test1: 0.3751811	best: 0.3751811 (627)	total: 53.6s	remaining: 31.7s
628:	learn: 0.3679846	test: 0.3679846	test1: 0.3751604	best: 0.3751604 (628)	total: 53.7s	remaining: 31.7s
629:	learn: 0.3679641	test: 0.3679641	test1: 0.3751416	best: 0.3751416 (629)	total: 53.8s	remaining: 31.6s
630:	learn: 0.3679419	test: 0.3679419	test1: 0.3751290	best: 0.3751290 (630)	total: 53.9s	remaining: 31.5s
631:	learn: 0.3677498	test: 0.3677498	test1: 0.3749646	best: 0.3749646 (631)	total: 54s	remaining: 31.4s
632:	learn: 0.3676460	test: 0.3676460	tes

701:	learn: 0.3657966	test: 0.3657966	test1: 0.3732903	best: 0.3732903 (701)	total: 59.8s	remaining: 25.4s
702:	learn: 0.3657818	test: 0.3657818	test1: 0.3732777	best: 0.3732777 (702)	total: 59.9s	remaining: 25.3s
703:	learn: 0.3657624	test: 0.3657624	test1: 0.3732626	best: 0.3732626 (703)	total: 1m	remaining: 25.2s
704:	learn: 0.3657422	test: 0.3657422	test1: 0.3732464	best: 0.3732464 (704)	total: 1m	remaining: 25.1s
705:	learn: 0.3657288	test: 0.3657288	test1: 0.3732356	best: 0.3732356 (705)	total: 1m	remaining: 25.1s
706:	learn: 0.3657113	test: 0.3657113	test1: 0.3732278	best: 0.3732278 (706)	total: 1m	remaining: 25s
707:	learn: 0.3656912	test: 0.3656912	test1: 0.3732226	best: 0.3732226 (707)	total: 1m	remaining: 24.9s
708:	learn: 0.3656690	test: 0.3656690	test1: 0.3732231	best: 0.3732226 (707)	total: 1m	remaining: 24.8s
709:	learn: 0.3656479	test: 0.3656479	test1: 0.3732160	best: 0.3732160 (709)	total: 1m	remaining: 24.7s
710:	learn: 0.3656423	test: 0.3656423	test1: 0.3732156	best:

781:	learn: 0.3638766	test: 0.3638766	test1: 0.3719392	best: 0.3719386 (780)	total: 1m 6s	remaining: 18.6s
782:	learn: 0.3638440	test: 0.3638440	test1: 0.3719090	best: 0.3719090 (782)	total: 1m 6s	remaining: 18.5s
783:	learn: 0.3636966	test: 0.3636966	test1: 0.3717866	best: 0.3717866 (783)	total: 1m 7s	remaining: 18.5s
784:	learn: 0.3636579	test: 0.3636579	test1: 0.3717561	best: 0.3717561 (784)	total: 1m 7s	remaining: 18.4s
785:	learn: 0.3636429	test: 0.3636429	test1: 0.3717397	best: 0.3717397 (785)	total: 1m 7s	remaining: 18.3s
786:	learn: 0.3636207	test: 0.3636207	test1: 0.3717253	best: 0.3717253 (786)	total: 1m 7s	remaining: 18.2s
787:	learn: 0.3636042	test: 0.3636042	test1: 0.3717185	best: 0.3717185 (787)	total: 1m 7s	remaining: 18.1s
788:	learn: 0.3635895	test: 0.3635895	test1: 0.3717156	best: 0.3717156 (788)	total: 1m 7s	remaining: 18s
789:	learn: 0.3635773	test: 0.3635773	test1: 0.3717039	best: 0.3717039 (789)	total: 1m 7s	remaining: 18s
790:	learn: 0.3635593	test: 0.3635593	tes

858:	learn: 0.3621654	test: 0.3621654	test1: 0.3706966	best: 0.3706966 (858)	total: 1m 13s	remaining: 12.1s
859:	learn: 0.3621492	test: 0.3621492	test1: 0.3706885	best: 0.3706885 (859)	total: 1m 13s	remaining: 12s
860:	learn: 0.3621382	test: 0.3621382	test1: 0.3706789	best: 0.3706789 (860)	total: 1m 13s	remaining: 11.9s
861:	learn: 0.3621204	test: 0.3621204	test1: 0.3706616	best: 0.3706616 (861)	total: 1m 13s	remaining: 11.8s
862:	learn: 0.3621049	test: 0.3621049	test1: 0.3706502	best: 0.3706502 (862)	total: 1m 13s	remaining: 11.7s
863:	learn: 0.3620800	test: 0.3620800	test1: 0.3706448	best: 0.3706448 (863)	total: 1m 14s	remaining: 11.7s
864:	learn: 0.3620606	test: 0.3620606	test1: 0.3706291	best: 0.3706291 (864)	total: 1m 14s	remaining: 11.6s
865:	learn: 0.3620451	test: 0.3620451	test1: 0.3706174	best: 0.3706174 (865)	total: 1m 14s	remaining: 11.5s
866:	learn: 0.3620294	test: 0.3620294	test1: 0.3706082	best: 0.3706082 (866)	total: 1m 14s	remaining: 11.4s
867:	learn: 0.3620179	test: 0.

936:	learn: 0.3607840	test: 0.3607840	test1: 0.3697800	best: 0.3697800 (935)	total: 1m 20s	remaining: 5.42s
937:	learn: 0.3607667	test: 0.3607667	test1: 0.3697718	best: 0.3697718 (937)	total: 1m 20s	remaining: 5.33s
938:	learn: 0.3607570	test: 0.3607570	test1: 0.3697650	best: 0.3697650 (938)	total: 1m 20s	remaining: 5.25s
939:	learn: 0.3607435	test: 0.3607435	test1: 0.3697570	best: 0.3697570 (939)	total: 1m 20s	remaining: 5.16s
940:	learn: 0.3607268	test: 0.3607268	test1: 0.3697415	best: 0.3697415 (940)	total: 1m 21s	remaining: 5.08s
941:	learn: 0.3607141	test: 0.3607141	test1: 0.3697277	best: 0.3697277 (941)	total: 1m 21s	remaining: 4.99s
942:	learn: 0.3606998	test: 0.3606998	test1: 0.3697179	best: 0.3697179 (942)	total: 1m 21s	remaining: 4.91s
943:	learn: 0.3606840	test: 0.3606840	test1: 0.3697040	best: 0.3697040 (943)	total: 1m 21s	remaining: 4.82s
944:	learn: 0.3606695	test: 0.3606695	test1: 0.3696927	best: 0.3696927 (944)	total: 1m 21s	remaining: 4.73s
945:	learn: 0.3605783	test: 

In [34]:
grid_search.best_params_

{'max_depth': 8, 'n_estimators': 1800}

In [36]:

    y_pred_train = grid_search.predict(X_train)
    y_pred_train[y_pred_train < 0] = 0
    y_pred_val = grid_search.predict(X_val)
    y_pred_val[y_pred_val < 0] = 0

    print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
    print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))
    print("RMS log-error train (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_train), np.expm1(y_pred_train))))
    print("RMS log-error val (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))))

RMS log-error train:  0.1364437391023863
RMS log-error val:  0.141150063024536
RMS log-error train (actual):  0.33814832447481663
RMS log-error val (actual):  0.35671279692695174


## Extra Trees

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor,AdaBoostRegressor

## Experiment V -- SVR

In [25]:
from sklearn.svm import SVR
run_experiment_V = False  # slow
if run_experiment_V:
    # svr = SVR(C=1.0, epsilon=0.2)
    svr = SVR(kernel='rbf')
    svr.fit(X_train, y_train.values.ravel())

In [26]:
if run_experiment_V:
    y_pred_train = svr.predict(X_train)
    y_pred_train[y_pred_train < 0] = 0
    y_pred_val = svr.predict(X_val)
    y_pred_val[y_pred_val < 0] = 0

    print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
    print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))
    print("RMS log-error train (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_train), np.expm1(y_pred_train))))
    print("RMS log-error val (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))))

## Test (Moment of Truth)

In [20]:
def main_predict(model, X_test):
    X_test_mod = X_test.copy()
    output = np.array([])
    start_day, end_day = X_test['day_of_month'].min(), X_test['day_of_month'].max()
        # we lost the dates, but we still have day_of_month, which is good enough for our experiment
        
    for day in range(start_day, end_day + 1):
        pred = model.predict(X_test_mod[X_test_mod['day_of_month'] == day])
        pred[pred < 0] = 0
        print(pred)
        output = np.concatenate([output, pred], axis=0)
        for future in range(day + 1, end_day + 1):
            X_test_mod.loc[X_test_mod[X_test_mod['day_of_month'] == future].index,
                           f'sales_lag_{(future - day):02d}'] = pred
            # fill out future values now that this sales figure is available
            
    return output

In [21]:
y_pred_test = main_predict(cat, X_test)

[1.67972993 0.06839461 1.70773033 ... 7.41113412 4.67156157 2.71448808]
[1.58746638 0.02633755 1.58720149 ... 7.09471576 4.75373818 2.73956356]
[1.59902703 0.01531816 1.55183311 ... 7.15424155 4.58767331 2.72389203]
[1.57575427 0.         1.50263062 ... 7.42416074 4.98467377 2.99020794]
[1.20155983 0.         1.23891777 ... 7.5285568  5.40594873 3.09785161]
[1.63064884 0.1250659  1.63510699 ... 7.30718999 4.97553467 2.78624159]
[1.53926037 0.03884906 1.44732587 ... 7.73851079 4.84598719 2.71878257]
[1.6000666  0.05421709 1.60586229 ... 7.14797204 4.55669643 2.68800279]
[1.49994938e+00 2.06997896e-03 1.42900014e+00 ... 7.07932879e+00
 4.72937603e+00 2.68593715e+00]
[1.53323609 0.00859577 1.40785548 ... 7.05022458 4.54724508 2.69407062]
[1.53166765 0.         1.53733868 ... 7.37536744 5.03096573 3.12499513]
[1.06626096 0.         1.17545285 ... 7.45399548 5.42690318 3.07231524]
[1.55252956 0.11851537 1.63454998 ... 7.24205607 5.03588246 2.63974857]
[1.50343136 0.04702078 1.60370649 ... 7

In [22]:
delta_index = 3008016 - 3000888  # we inserted 4 Christmas days, 4 x 54 x 33 = 7128, which is the difference
#submission = pd.DataFrame({'id': X_test.index - delta_index, 'sales': np.expm1(y_pred_test)})
#submission = pd.DataFrame({'id': X_test.index - delta_index, 'sales': max(y_pred_test, 0)})
submission = pd.DataFrame({'id': X_test.index - delta_index, 'sales': np.expm1(y_pred_test)})
submission.to_csv('submissioncat.csv', index=False)